# Sentiment analysis with Word2Vec

### Exercise objectives:
- Convert words to vectors with Word2Vec
- Use the word representation given by Word2vec to feed a RNN

<hr>

# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [1]:
import numpy as np

In [2]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2022-11-18 12:39:52.263213: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In the previous exercise, you trained a Word2vec representation and converted all your training sentences in order to feed them into a RNN, as shown in the first step of this Figure: 

<img src="word2vec_representation.png" width="400px" />



❓ **Question** ❓ Here, let's re-do exactly what you have done in the previous exercise. First, train a word2vec model (with the arguments that you want) on your training sentence. Store it into the `word2vec` variable.

In [3]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train, vector_size=100, min_count=3, window=10)

Let's reuse the functions of the previous exercise to convert your training and test data into something you can feed into a RNN.

❓ **Question** ❓ Read the following function to be sure you understand what is going on, and run it.

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

☝️ To be sure that it worked, let's check the following for `X_train_pad` and `X_test_pad`:
- they are numpy arrays
- they are 3-dimensional
- the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`
- the first dimension is of the size of your `X_train` and `X_test`

✅ **Good Practice** ✅ Such tests are quite important! Not only in this exercise, but in real-life applications. It prevents from finding errors too late and from letting them propagate through the entire notebook.

In [5]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

# Baseline model

It is always good to have a very simple model to test your own model against - to be sure you are doing something better than a very simple algorithm.

❓ **Question** ❓ What is your baseline accuracy? In this case, your baseline can be to predict the label that is the most present in `y_train` (of course, if the dataset is balanced, the baseline accuracy is 1/n where n is the number of classes - 2 here).

In [15]:
print(len(y_train))
print(y_train.sum())

baseline_accuracy = 1/2

2500
1235


# The model

❓ **Question** ❓ Write a RNN with the following layers:
- a `Masking` layer
- a `LSTM` with 20 units and `tanh` activation function
- a `Dense` with 10 units
- an output layer that depends on your task

Then, compile your model (we advise you to use the `rmsprop` as the optimizer - at least to begin with)

In [37]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


model = Sequential()
model.add(layers.Masking(mask_value=0)) # masking
model.add(layers.LSTM(units=20, activation='tanh'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# –– Compilation
model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop',
              metrics='accuracy') 

# –– Fit
model.fit(X_train_pad, y_train,
         validation_split=0.3,
         callbacks=EarlyStopping(patience=10),
         batch_size=16,
         epochs=50);

Epoch 1/50
110/110 [==============================] - 6s 35ms/step - loss: 0.6919 - accuracy: 0.5240 - val_loss: 0.6837 - val_accuracy: 0.5720
Epoch 2/50
110/110 [==============================] - 3s 29ms/step - loss: 0.6774 - accuracy: 0.5594 - val_loss: 0.6925 - val_accuracy: 0.5493
Epoch 3/50
110/110 [==============================] - 3s 29ms/step - loss: 0.6617 - accuracy: 0.6080 - val_loss: 0.6643 - val_accuracy: 0.6107
Epoch 4/50
110/110 [==============================] - 3s 29ms/step - loss: 0.6456 - accuracy: 0.6240 - val_loss: 0.6523 - val_accuracy: 0.6347
Epoch 5/50
110/110 [==============================] - 3s 29ms/step - loss: 0.6254 - accuracy: 0.6531 - val_loss: 0.6580 - val_accuracy: 0.6133
Epoch 6/50
110/110 [==============================] - 3s 29ms/step - loss: 0.6199 - accuracy: 0.6594 - val_loss: 0.6801 - val_accuracy: 0.5880
Epoch 7/50
110/110 [==============================] - 3s 29ms/step - loss: 0.6021 - accuracy: 0.6840 - val_loss: 0.6411 - val_accuracy: 0.6333

❓ **Question** ❓ Fit the model on your embedded and padded data - do not forget the early stopping criterion.

❗ **Remark** ❗ Your accuracy will greatly depend on your training corpus. Here just make sure that your performance is above the baseline model (which should be the case even if you loaded only 20% of the initial IMDB data).

❓ **Question** ❓ Evaluate your model on the test set

In [22]:
model.evaluate(X_test_pad, y_test)

79/79 [==============================] - 0s 5ms/step - loss: 0.6328 - accuracy: 0.6476


[0.6327817440032959, 0.647599995136261]

# Trained Word2Vec - Transfer Learning

Your accuracy, while above the baseline model, might be quite low. There are multiple options to improve it, as data cleaning and improving the quality of the embedding.

We won't dig into data cleaning strategies here. Let's try to improve the quality of our embedding. But instead of just loading a larger corpus, why not benefiting from the embedding that others have learned? Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.



❓ **Question** ❓ List all the different models available in the word2vec thanks to this: 

In [23]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


ℹ️ You can also find the list of the models and their size on the [`gensim-data` repository](https://github.com/RaRe-Technologies/gensim-data#models).

❓ **Question** ❓ Load one of the pre-trained word2vec embedding spaces. 

You can do that with `api.load(the-model-of-your-choice)`, and store it in `word2vec_transfer`

<details>
    <summary>💡 Hint</summary>
    
The `glove-wiki-gigaword-50` model is a good candidate to start with as it is smaller (65 MB).

</details>

In [26]:
import gensim.downloader as api

word2vec_transfer = api.load("glove-wiki-gigaword-50")

[===================-------------------------------] 38.6% 25.5/66.0MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==============================================----] 92.7% 61.2/66.0MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



❓ **Question** ❓ Check the size of the vocabulary, but also the size of the embedding space.

In [32]:
# Size of the vocabulary
print(len(word2vec_transfer))

# Size of the embedding space
print(len(word2vec_transfer[0]))

400000
50


❓ Let's embed `X_train` and `X_test`, same as in the first question where we provided the functions to do so! (There is a slight difference in the `embed_sentence_with_TF` function that we will not dig into)

In [33]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

❓ **Question** ❓  Do not forget to pad your results and store it in `X_train_pad_2` and `X_test_pad_2`.

In [38]:
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

❓ **Question** ❓ Reinitialize a model and fit it on your new embedded (and padded) data!  Evaluate it on your test set and compare it to your previous accuracy.

❗ **Remark** ❗ The training here could take some time. You can just compute 10 epochs (this is **not** a good practice, it is just not to wait too long) and go to the next exercise while it trains - or take a break, you probably deserve it ;)

In [39]:
model = Sequential()
model.add(layers.Masking(mask_value=0)) # masking
model.add(layers.LSTM(units=2, activation='tanh'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# –– Compilation
model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop',
              metrics='accuracy') 

# –– Fit
model.fit(X_train_pad_2, y_train,
         validation_split=0.3,
         callbacks=EarlyStopping(patience=5),
         batch_size=16,
         epochs=10);

Epoch 1/10
110/110 [==============================] - 6s 29ms/step - loss: 0.6930 - accuracy: 0.5109 - val_loss: 0.6894 - val_accuracy: 0.5427
Epoch 2/10
110/110 [==============================] - 3s 24ms/step - loss: 0.6894 - accuracy: 0.5371 - val_loss: 0.6847 - val_accuracy: 0.5773
Epoch 3/10
110/110 [==============================] - 3s 25ms/step - loss: 0.6857 - accuracy: 0.5520 - val_loss: 0.6799 - val_accuracy: 0.5853
Epoch 4/10
110/110 [==============================] - 3s 26ms/step - loss: 0.6812 - accuracy: 0.5651 - val_loss: 0.6737 - val_accuracy: 0.5960
Epoch 5/10
110/110 [==============================] - 3s 24ms/step - loss: 0.6759 - accuracy: 0.5783 - val_loss: 0.6733 - val_accuracy: 0.5827
Epoch 6/10
110/110 [==============================] - 3s 25ms/step - loss: 0.6707 - accuracy: 0.5886 - val_loss: 0.6659 - val_accuracy: 0.6013
Epoch 7/10
110/110 [==============================] - 3s 24ms/step - loss: 0.6649 - accuracy: 0.5960 - val_loss: 0.6614 - val_accuracy: 0.6093

In [40]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 60.000%


Because your new word2vec has been trained on a large corpus, it has a representation for many many words! Way more than with your small dataset, especially as you discarded words that were not present more than a given number of times in the train set. For that reason, you have way more embedded words in your train and test set, which makes each iteration longer than previously